# Corridor Generation Visualization

Interactive 3D visualization of flyable corridors generated by `generate_corridors.py`.

In [1]:
import json
import subprocess
import numpy as np
import plotly.graph_objects as go
from typing import List, Dict

def generate_corridors(grid_size=400, num_ew=2, num_ns=2, width="20", 
                       spacing="100", altitude_min=0, altitude_max=400, seed=None):
    """Run generate_corridors.py and return parsed corridors."""
    cmd = [
        "python3", "generate_corridors.py",
        "--grid-size", str(grid_size),
        "--num-ew", str(num_ew),
        "--num-ns", str(num_ns),
        "--width", str(width),
        "--spacing", str(spacing),
        "--altitude-min", str(altitude_min),
        "--altitude-max", str(altitude_max),
    ]
    if seed is not None:
        cmd.extend(["--seed", str(seed)])
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    corridors = [json.loads(line) for line in result.stdout.strip().split('\n') if line]
    return corridors


def create_corridor_mesh(corridor: Dict, grid_size: float, color: str, opacity: float = 0.3):
    """Create a 3D mesh for a corridor."""
    c = corridor["center"]
    w = corridor["width"] / 2
    z_min = corridor["altitude_min"]
    z_max = corridor["altitude_max"]
    
    if corridor["direction"] == "EW":
        # Runs along X axis, varies in Y
        x = [0, grid_size, grid_size, 0, 0, grid_size, grid_size, 0]
        y = [c-w, c-w, c+w, c+w, c-w, c-w, c+w, c+w]
    else:  # NS
        # Runs along Y axis, varies in X
        x = [c-w, c+w, c+w, c-w, c-w, c+w, c+w, c-w]
        y = [0, 0, grid_size, grid_size, 0, 0, grid_size, grid_size]
    
    z = [z_min, z_min, z_min, z_min, z_max, z_max, z_max, z_max]
    
    # Define the 6 faces of the cuboid
    i = [0, 0, 4, 4, 0, 1, 0, 3, 1, 2, 4, 5]
    j = [1, 2, 5, 6, 1, 5, 3, 7, 2, 6, 5, 6]
    k = [2, 3, 6, 7, 4, 4, 4, 4, 5, 5, 1, 2]
    
    return go.Mesh3d(
        x=x, y=y, z=z,
        i=i, j=j, k=k,
        color=color,
        opacity=opacity,
        name=f"{corridor['direction']} corridor {corridor['id']}",
        hovertemplate=(
            f"ID: {corridor['id']}<br>"
            f"Direction: {corridor['direction']}<br>"
            f"Center: {corridor['center']:.1f}m<br>"
            f"Width: {corridor['width']:.1f}m<br>"
            f"Altitude: {z_min:.0f}-{z_max:.0f}m"
            "<extra></extra>"
        )
    )


def visualize_corridors(corridors: List[Dict], grid_size: float = 400, title: str = "Corridors"):
    """Create interactive 3D visualization of corridors."""
    fig = go.Figure()
    
    # Add corridors
    for corridor in corridors:
        color = "blue" if corridor["direction"] == "EW" else "red"
        fig.add_trace(create_corridor_mesh(corridor, grid_size, color))
    
    # Add ground plane
    fig.add_trace(go.Mesh3d(
        x=[0, grid_size, grid_size, 0],
        y=[0, 0, grid_size, grid_size],
        z=[0, 0, 0, 0],
        i=[0, 0], j=[1, 2], k=[2, 3],
        color="lightgray",
        opacity=0.5,
        name="Ground",
        hoverinfo="skip"
    ))
    
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title="X (East) [m]",
            yaxis_title="Y (North) [m]",
            zaxis_title="Altitude [m]",
            aspectmode="data",
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.0))
        ),
        legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
        margin=dict(l=0, r=0, t=40, b=0),
        height=600
    )
    
    return fig

## Example 1: Default Configuration

2 East-West and 2 North-South corridors with constant width (20m) and spacing (100m).

In [2]:
corridors = generate_corridors(seed=42)
for c in corridors:
    print(json.dumps(c))

fig = visualize_corridors(corridors, title="Default: 2 EW + 2 NS, width=20m, spacing=100m")
fig.show()

{"id": 0, "direction": "EW", "center": 100.0, "width": 20.0, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 1, "direction": "EW", "center": 200.0, "width": 20.0, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 2, "direction": "NS", "center": 100.0, "width": 20.0, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 3, "direction": "NS", "center": 200.0, "width": 20.0, "altitude_min": 0.0, "altitude_max": 400.0}


## Example 2: Single Intersection

1 EW and 1 NS corridor - shows how few corridors cluster near the origin.

In [3]:
corridors = generate_corridors(num_ew=1, num_ns=1, spacing="150", seed=42)
for c in corridors:
    print(json.dumps(c))

fig = visualize_corridors(corridors, title="Single Intersection: 1 EW + 1 NS, spacing=150m")
fig.show()

{"id": 0, "direction": "EW", "center": 150.0, "width": 20.0, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 1, "direction": "NS", "center": 150.0, "width": 20.0, "altitude_min": 0.0, "altitude_max": 400.0}


## Example 3: Dense Grid

More corridors with tighter spacing.

In [4]:
corridors = generate_corridors(num_ew=4, num_ns=4, width="15", spacing="80", seed=42)
for c in corridors:
    print(json.dumps(c))

fig = visualize_corridors(corridors, title="Dense Grid: 4 EW + 4 NS, width=15m, spacing=80m")
fig.show()

{"id": 0, "direction": "EW", "center": 80.0, "width": 15.0, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 1, "direction": "EW", "center": 160.0, "width": 15.0, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 2, "direction": "EW", "center": 240.0, "width": 15.0, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 3, "direction": "EW", "center": 320.0, "width": 15.0, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 4, "direction": "NS", "center": 80.0, "width": 15.0, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 5, "direction": "NS", "center": 160.0, "width": 15.0, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 6, "direction": "NS", "center": 240.0, "width": 15.0, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 7, "direction": "NS", "center": 320.0, "width": 15.0, "altitude_min": 0.0, "altitude_max": 400.0}


## Example 4: Randomized Width and Spacing

Corridors with varying widths and irregular spacing.

In [5]:
corridors = generate_corridors(num_ew=3, num_ns=3, width="15-30", spacing="70-120", seed=42)
for c in corridors:
    print(json.dumps(c))

fig = visualize_corridors(corridors, title="Randomized: width=15-30m, spacing=70-120m, seed=42")
fig.show()

{"id": 0, "direction": "EW", "center": 101.97133992289419, "width": 15.375161328340004, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 1, "direction": "EW", "center": 185.72280584135015, "width": 18.34816107223234, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 2, "direction": "EW", "center": 292.5463665495508, "width": 25.15049231134367, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 3, "direction": "NS", "center": 114.60897838524227, "width": 16.304082489441242, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 4, "direction": "NS", "center": 205.7050693695058, "width": 15.446958291571056, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 5, "direction": "NS", "center": 286.636968109686, "width": 22.580329321550437, "altitude_min": 0.0, "altitude_max": 400.0}


## Example 5: Same Parameters, Different Seeds

Demonstrates how different seeds produce different corridor layouts.

In [6]:
from plotly.subplots import make_subplots

# Generate corridors with different seeds
configs = [
    {"seed": 1, "title": "Seed 1"},
    {"seed": 42, "title": "Seed 42"},
    {"seed": 99, "title": "Seed 99"},
]

for cfg in configs:
    corridors = generate_corridors(num_ew=2, num_ns=2, width="15-25", spacing="80-120", seed=cfg["seed"])
    print(f"\n=== {cfg['title']} ===")
    for c in corridors:
        print(f"  {c['direction']} @ {c['center']:.1f}m, width={c['width']:.1f}m")
    
    fig = visualize_corridors(corridors, title=f"Randomized Layout - {cfg['title']}")
    fig.show()


=== Seed 1 ===
  EW @ 85.4m, width=23.5m
  EW @ 195.9m, width=17.6m
  NS @ 99.8m, width=19.5m
  NS @ 205.9m, width=22.9m



=== Seed 42 ===
  EW @ 105.6m, width=15.3m
  EW @ 196.6m, width=17.2m
  NS @ 109.5m, width=21.8m
  NS @ 225.1m, width=15.9m



=== Seed 99 ===
  EW @ 96.2m, width=17.0m
  EW @ 183.3m, width=17.5m
  NS @ 110.4m, width=17.5m
  NS @ 205.7m, width=21.8m


## Example 6: Limited Altitude Range

Corridors with restricted altitude band (simulating low-altitude urban flight).

In [7]:
corridors = generate_corridors(
    num_ew=2, num_ns=2, 
    width="25", spacing="120",
    altitude_min=30, altitude_max=100,
    seed=42
)
for c in corridors:
    print(json.dumps(c))

fig = visualize_corridors(corridors, title="Limited Altitude: 30-100m")
fig.show()

{"id": 0, "direction": "EW", "center": 120.0, "width": 25.0, "altitude_min": 30.0, "altitude_max": 100.0}
{"id": 1, "direction": "EW", "center": 240.0, "width": 25.0, "altitude_min": 30.0, "altitude_max": 100.0}
{"id": 2, "direction": "NS", "center": 120.0, "width": 25.0, "altitude_min": 30.0, "altitude_max": 100.0}
{"id": 3, "direction": "NS", "center": 240.0, "width": 25.0, "altitude_min": 30.0, "altitude_max": 100.0}


## Example 7: Asymmetric Layout

More EW corridors than NS (e.g., simulating a city with major East-West avenues).

In [8]:
corridors = generate_corridors(num_ew=4, num_ns=1, width="20", spacing="70", seed=42)
for c in corridors:
    print(json.dumps(c))

fig = visualize_corridors(corridors, title="Asymmetric: 4 EW + 1 NS")
fig.show()

{"id": 0, "direction": "EW", "center": 70.0, "width": 20.0, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 1, "direction": "EW", "center": 140.0, "width": 20.0, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 2, "direction": "EW", "center": 210.0, "width": 20.0, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 3, "direction": "EW", "center": 280.0, "width": 20.0, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 4, "direction": "NS", "center": 70.0, "width": 20.0, "altitude_min": 0.0, "altitude_max": 400.0}


## Example 8: Large Grid with Sparse Corridors

Shows how few corridors cluster near the origin in a large grid.

In [9]:
corridors = generate_corridors(
    grid_size=1000,
    num_ew=2, num_ns=2, 
    width="30", spacing="150",
    seed=42
)
for c in corridors:
    print(json.dumps(c))

fig = visualize_corridors(corridors, grid_size=1000, title="Large Grid (1000m): Corridors cluster near origin")
fig.show()

{"id": 0, "direction": "EW", "center": 150.0, "width": 30.0, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 1, "direction": "EW", "center": 300.0, "width": 30.0, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 2, "direction": "NS", "center": 150.0, "width": 30.0, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 3, "direction": "NS", "center": 300.0, "width": 30.0, "altitude_min": 0.0, "altitude_max": 400.0}


## Example 9: Grid Boundary Behavior

Requesting more corridors than can fit - shows warning and truncation.

In [10]:
# Request 10 corridors but only ~3-4 will fit with spacing=100 in 400m grid
corridors = generate_corridors(num_ew=10, num_ns=0, width="20", spacing="100", seed=42)
print(f"Requested 10 EW corridors, got {len(corridors)}")
for c in corridors:
    print(json.dumps(c))

fig = visualize_corridors(corridors, title="Boundary Truncation: Requested 10, got fewer")
fig.show()

Requested 10 EW corridors, got 3
{"id": 0, "direction": "EW", "center": 100.0, "width": 20.0, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 1, "direction": "EW", "center": 200.0, "width": 20.0, "altitude_min": 0.0, "altitude_max": 400.0}
{"id": 2, "direction": "EW", "center": 300.0, "width": 20.0, "altitude_min": 0.0, "altitude_max": 400.0}
